In [1]:
# 저장 된 비디오를 읽고 3d pose estimation 이 된 frame을 다시 비디오로 저장.

import os
import sys

import cv2
from PIL import Image
import numpy as np
import torch
import torchvision

model_path = os.path.abspath(os.path.join('..'))
if model_path not in sys.path:
    sys.path.append(model_path)

from colorhandpose3d.model.ColorHandPose3D import ColorHandPose3D
from colorhandpose3d.model.HandNet import HandNet
from colorhandpose3d.utils.general import *
from colorhandpose3d.utils.transforms import *


from matplotlib import pyplot as plt
from tqdm import tqdm
import time

# Initialize models and load weights
weight_path = "../saved/"
use_cuda = torch.cuda.is_available()
chp3d = ColorHandPose3D(weight_path, crop_size=224)

if use_cuda:
    chp3d.cuda()




# Initialize OpenCV object
video_name = 'test_easy.avi'
cap = cv2.VideoCapture(os.path.join("../outputs/in_video/", video_name))  # Open video
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(frame_width, frame_height, length, end=",")

# Define image transforms
transform0 = torchvision.transforms.ToPILImage()
transform1 = torchvision.transforms.ToTensor()
transform2 = torchvision.transforms.Resize((224, 224))
transform3 = torchvision.transforms.Normalize(mean=[0.641, 0.619, 0.612], std=[0.254, 0.262, 0.258])
transform4 = torchvision.transforms.Resize((240, 320))
preprocess_chp3d = torchvision.transforms.Compose([transform0, transform2, transform1])
preprocess_handnet = torchvision.transforms.Compose([transform1, transform3])

def main():
    import gc
    # Preprocessing recorded frames
    frames = []
    i = 0
    while cap.isOpened():
        ret, frame = cap.read()  # frame: numpy array, (H480, W640, C3)

        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB
            sample_original = preprocess_chp3d(frame).unsqueeze(0)
#             sample_original = transform1(frame).unsqueeze(0)  # (C3, H480, W640)
            sample_chp = sample_original - 0.5
            sample_handnet = preprocess_handnet(frame)
            sample_handnet = sample_handnet.unsqueeze(0)
            hand_side = torch.tensor([[1.0, 0.0]])

#             if use_cuda:
#                 sample = sample.cuda()
#                 hand_side = hand_side.cuda()

            frames.append([sample_handnet, sample_chp, sample_original, hand_side])
            _frame = transform0(frame)
            _frame.save("../outputs/in_video/{0:04d}.png".format(i+1))
            i += 1

        
            del sample_original
            del sample_chp
            del sample_handnet
            del hand_side
            gc.collect()
            print(i)

        else:
            break

    # Pass frames through chp3d
    passFrames(frames)

    cap.release()
    cv2.destroyAllWindows()


def passFrames(frames):

    # Video writer
#     fourcc = cv2.VideoWriter_fourcc(*"DIVX")
#     out = cv2.VideoWriter("../outputs/after_test.avi", fourcc, 25.0, (640, 480))

    for i, frame in tqdm(enumerate(frames)):
        sample_handnet = frame[0]
        sample_chp = frame[1]
        sample_original = frame[2]
        hand_side = frame[3]
        
        if use_cuda:
            sample_handnet = sample_handnet.cuda()
            sample_chp = sample_chp.cuda()
            hand_side = hand_side.cuda()
            
        
        # Pass frame through chp3d
        inputs = [sample_handnet, sample_chp]
        s = time.time()
        coords_xyz_rel_normed, keypoint_scoremap, image_crop, centers, scale_crop, hand_mask = chp3d(
            inputs, hand_side, deploy=True
        )
        print('Time for processing 1 frame: {}s'.format(time.time() - s))
        

        # hand flag
        if coords_xyz_rel_normed is None:
#             img = transform4(transform0(sample_original.squeeze()))
#             fig = plt.figure()
#             plt.imshow(img)
#             plt.axis("off")
#             plt.tight_layout()
#             plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
#             pil = fig2img(fig)
#             pil.save("../outputs/out_video/ver0_3/{0:04d}.png".format(i+1))
# #             plt.savefig("../outputs/out_video/ver0_3/{0:04d}.png".format(i+1))
#             plt.close()
    
            
            # save 3d
            fig = plt.figure(1, figsize=(16, 16))
            ax1 = fig.add_subplot(111, projection='3d')
            ax1.view_init(azim=-90, elev=-90)
            ax1.set_xlim([-3, 3])
            ax1.set_ylim([-4, 4])
            ax1.set_zlim([-5, 5])
            plt.savefig("../outputs/out_video/ver0_3/3d/{0:04d}.png".format(i+1))
            plt.close()

            
        else:
            # Back to CPU
            if use_cuda is True:
                coords_xyz_rel_normed = coords_xyz_rel_normed.cpu()
                keypoint_scoremap = keypoint_scoremap.cpu()
                image_crop = image_crop.cpu()
                centers = centers.cpu()
                scale_crop = scale_crop.cpu()
                hand_mask = hand_mask.cpu()

            keypoint_coords3d = coords_xyz_rel_normed.detach().numpy()
            keypoint_coords3d = keypoint_coords3d.squeeze()
            
            ## save 3d 
            fig = plt.figure(1, figsize=(16, 16))
            ax1 = fig.add_subplot(111, projection='3d')
            plot_hand_3d(keypoint_coords3d, ax1)
            ax1.view_init(azim=-90, elev=-90)
            ax1.set_xlim([-3, 3])
            ax1.set_ylim([-4, 4])
            ax1.set_zlim([-5, 5])
            plt.savefig("../outputs/out_video/ver0_3/3d/{0:04d}.png".format(i+1))
            plt.close()


#             keypoint_coords_crop = detect_keypoints(keypoint_scoremap[0].detach().numpy())
#             keypoint_coords = transform_cropped_coords(keypoint_coords_crop, centers, scale_crop, 224)


#             img = transform0(sample_original.squeeze())
            
#             fig = plt.figure()
#             plt.imshow(img)
#             plot_hand(keypoint_coords, plt)
#             plt.axis("off")
#             plt.tight_layout()
#             plt.subplots_adjust(left = 0, bottom = 0, right = 1, top = 1, hspace = 0, wspace = 0)
#             pil = fig2img(fig)
#             pil = transform4(pil)
#             pil.save("../outputs/out_video/ver0_3/{0:04d}.png".format(i+1))

#             plt.close()

320.0 240.0 1250,

In [2]:
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf, bbox_inches='tight', pad_inches=0)
    buf.seek(0)
    img = Image.open(buf)
    return img

In [ ]:
main()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


0it [00:00, ?it/s]

1244
1245
1246
1247
1248
1249
tensor([[0.0000]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.01228189468383789
no hand
Time for processing 1 frame: 0.01244044303894043s


2it [00:00,  7.21it/s]

tensor([[0.0000]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.009752273559570312
no hand
Time for processing 1 frame: 0.009928226470947266s
tensor([[0.0000]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.01088261604309082
no hand
Time for processing 1 frame: 0.011202096939086914s


...

tensor([[0.9997]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.007757663726806641
Handsegnet forward time: 0.0010666847229003906
s shape torch.Size([1, 2, 224, 224])
for loop time 0.0034372806549072266
hand mask sum: 3982.0
Calculate single highest scoring object time: 0.030545949935913086
Crop and resize time: 0.000982522964477539
Posenet forward time: 0.0019714832305908203
Posepriornet forward time: 0.0007550716400146484
Viewpoint forward time: 0.0009315013885498047
Get normalized 3d coordinates time: 0.0006077289581298828
Flip left handed input time: 0.00013399124145507812
ScaLe heatmaps time 0.00010824203491210938
Time for processing 1 frame: 0.04549050331115723s


1001it [03:38,  4.13it/s]

tensor([[1.0000]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.008272171020507812
Handsegnet forward time: 0.0010237693786621094
s shape torch.Size([1, 2, 224, 224])
for loop time 0.0031440258026123047
hand mask sum: 3959.0
Calculate single highest scoring object time: 0.03075432777404785
Crop and resize time: 0.0010805130004882812
Posenet forward time: 0.0020105838775634766
Posepriornet forward time: 0.0007250308990478516
Viewpoint forward time: 0.0009286403656005859
Get normalized 3d coordinates time: 0.0008730888366699219
Flip left handed input time: 0.0002231597900390625
ScaLe heatmaps time 4.792213439941406e-05
Time for processing 1 frame: 0.04674029350280762s


1002it [03:38,  4.09it/s]

tensor([[1.0000]], device='cuda:0', grad_fn=<SigmoidBackward>)
Handnet forward time: 0.11805295944213867
Handsegnet forward time: 0.5411152839660645
s shape torch.Size([1, 2, 224, 224])
for loop time 0.05636930465698242
hand mask sum: 3918.0
Calculate single highest scoring object time: 1.7731819152832031
Crop and resize time: 0.43250584602355957
